In [13]:
import java.io.File
import kotlinx.serialization.json.*
import io.github.lexadiky.jsonquery.*

val json = Json.parseToJsonElement(
    File("dataset.json").readText()
)

In [14]:
// Using path
json.query { path("shelter.name") }

"Purrfect Paws Cat Shelter"

In [15]:
// Using path with index
println(json.query { path("shelter.cats")[0].path("name") })
println(json.query { path("shelter.cats.0.name") })
println(json.query("shelter.cats.0.name"))

"Mittens"
"Mittens"
"Mittens"


In [16]:
// Using path with in array lookup
json.query { path("shelter.cats.name") }

["Mittens","Shadow","Mumpkin"]

In [17]:
// Using path with filter
json.query { path("shelter.cats.name").filterT<String> { it.startsWith("M") } }

["Mittens","Mumpkin"]

In [18]:
// Slices
json.query { path("shelter.cats")[0..1].path("name") }

["Mittens","Shadow"]

In [19]:
// Statistical operations
println(json.query { path("shelter.cats.age").max() })
println(json.query { path("shelter.cats.age").min() })
println(json.query { path("shelter.cats.age").average() })
println(json.query { path("shelter.cats.age").sum() })
println(json.query { path("shelter.cats.age").first() })
println(json.query { path("shelter.cats.age").last() })
println(json.query { path("shelter.cats.age").filterT<Int> { it >= 2 }.size() })

3.0
1.0
2.0
6.0
3
1
2


In [20]:
// Selecting specific fields
println(json.query { path("shelter.cats.0").select("name", "age") })
println(json.query { path("shelter.cats").select("name", "age") })

{"name":"Mittens","age":3}
[{"name":"Mittens","age":3},{"name":"Shadow","age":2},{"name":"Mumpkin","age":1}]


In [21]:
// Aggregating data into object
json.query {
    path("shelter.cats").qmap {
        buildJsonObject {
            put("names", query { path("name") })
            put("ages", query { path("age") })
        }
    }
}

{"names":["Mittens","Shadow","Mumpkin"],"ages":[3,2,1]}

In [22]:
// Mapping
val baseQuery = json.query("shelter.cats.0.name")

println(baseQuery.query { map { JsonPrimitive(it.jsonPrimitive.content.length) } })
println(baseQuery.query { mapT<String, Int> { it.length } })

7
7


In [23]:
// On Each
json.query {
    path("shelter.cats.name").each {
        mapT<String, Int> { it.length }
    }
}

[7,6,7]

In [24]:
// List and map type conversions
println(
    json.query {
        path("shelter.cats.age")
            .mapT<List<String>, String> { it.joinToString(", ") }
    }
)

println(
    json.query {
        path("shelter.cats.0")
            .mapT<Map<String, String>, List<String>> { it.keys.toList() }
    }
)

"3, 2, 1"
["id","name","age","breed","color","gender","adopted","medicalRecords","favoriteToys","friends"]
